In [1]:
import pandas as pd
import json
import json
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append("../src")
import prompt_utils
import os
import random
from typing import List

# vicuna 
# with rules classification only (0.76) (0.7-0.8s per prompt)
vicuna_base_path = "../data/vicuna_4bit/"
vicuna_with_rules_classification_only_name = "generic_prompt_with_rules_only_classification"
vicuna_with_rules_classification_only_func = prompt_utils.get_vicuna_prompt_with_rules_only_classification

# OA LLAMA
# Classification Only V03 (0.81)
# 1 pos 1 neg (0.79)
oa_base_path = "../data/openassistant_llama_30b_4bit/"
oa_classification_only_v03_name = "generic_prompt_without_context_only_classification_v03"
oa_classification_only_v03_func = prompt_utils.get_openassistant_llama_30b_4bit_without_context_only_classification_v03
oa_1pos_1neg_path_name = "generic_prompt_few_shot_prompt_only_classification_1_pos_1_neg_example"
oa_1pos_1neg_path_func = prompt_utils.get_openassistant_llama_30b_4bit_few_shot_prompt_only_classification_1_pos_1_neg_example

# Text Davinci
# Elaboration First V02 (0.94)
davinci_base_path = "../data/openai_text_davinci_003/"
davinci_elaboration_first_v02_name = "generic_prompt_without_context_elaboration_first_v02"
davinci_elaboration_first_v02_func = prompt_utils.get_openai_prompt_without_context_elaboration_first_v02

# Define a list of filenames to load
labeled_data_filename = "../data/labeled_data/generic_test_0.json"

dfs = []
with open(labeled_data_filename) as f:
    data = json.load(f)
df = pd.DataFrame(data["train"])
dfs.append(df)
df = pd.DataFrame(data["test"])
dfs.append(df)
df = pd.DataFrame(data["valid"])
dfs.append(df)
df_all = pd.concat(dfs)
ALL_LABELS = prompt_utils.ALL_LABELS[:-1]
LOW_F1_LABELS = prompt_utils.LOW_F1_LABELS

In [2]:
def initialize_eng_dataframe():
    unlabeled_dataset = prompt_utils.generate_unlabeled_dataset()
    return unlabeled_dataset.loc[unlabeled_dataset.tweet_language == "en"].sample(frac=1, random_state = 42).reset_index()

def find_start_index(df: pd.DataFrame, new_column_names: List[str]) -> int:
    # Find first row where all weak labels are NaN
    try:
        start_index = df[new_column_names].isna().all(axis=1).idxmax()
        # If no NaN found in any of new_column_names, start from beginning
        if pd.isna(start_index):
            start_index = 0
        return start_index
    except:
        return 0

In [3]:
output_folder = f"{vicuna_base_path}/{vicuna_with_rules_classification_only_name}"
model_name = "low_f1_labels_weak_labeling"
prompt_func = vicuna_with_rules_classification_only_func
idx_of_rules_of_low_f1_labels = [1, 2, 4, 9, 12, 13]

if os.path.isfile(f"{output_folder}/{model_name}.csv"):
    eng_tweets = pd.read_csv(f"{output_folder}/{model_name}.csv")
else:
    eng_tweets = initialize_eng_dataframe()
    eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)

/tmp/ipykernel_88926/3980851207.py:7: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,19,20,22,23,28,29,30,32,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  eng_tweets = pd.read_csv(f"{output_folder}/{model_name}.csv")


In [5]:
import numpy as np

mask_conspiracy = eng_tweets['Conspiracy Theory_pred'].str.contains('0', na=False)
mask_education = eng_tweets['Education_pred'].str.contains('0', na=False)
mask_environment = eng_tweets['Environment_pred'].str.contains('0', na=False)
mask_labor = eng_tweets['Labor/Employment_pred'].str.contains('0', na=False)
mask_religion = eng_tweets['Religion_pred'].str.contains('0', na=False)
mask_science = eng_tweets['Science/Technology_pred'].str.contains('0', na=False)

conspiracy_preds = eng_tweets[eng_tweets['Conspiracy Theory_pred'].str.contains('1', na=False)]
print("Amount of Conspiracy Theory predictions: ", len(conspiracy_preds))

only_conspiracy_predicted = eng_tweets[eng_tweets['Conspiracy Theory_pred'].str.contains('1', na=False) & mask_education & mask_environment & mask_labor & mask_religion & mask_science]
print("Amount of Conspiracy Theory predictions with all other predictions being 0: ", len(only_conspiracy_predicted))

education_preds = eng_tweets[eng_tweets['Education_pred'].str.contains('1', na=False)]
print("Amount of Education predictions: ", len(education_preds))

only_education_predicted = eng_tweets[eng_tweets['Education_pred'].str.contains('1', na=False) & mask_conspiracy & mask_environment & mask_labor & mask_religion & mask_science]
print("Amount of Education predictions with all other predictions being 0: ", len(only_education_predicted))

environment_preds = eng_tweets[eng_tweets['Environment_pred'].str.contains('1', na=False)]
print("Amount of Environment predictions: ", len(environment_preds))

only_environment_predicted = eng_tweets[eng_tweets['Environment_pred'].str.contains('1', na=False) & mask_conspiracy & mask_education & mask_labor & mask_religion & mask_science]
print("Amount of Environment predictions with all other predictions being 0: ", len(only_environment_predicted))

labor_preds = eng_tweets[eng_tweets['Labor/Employment_pred'].str.contains('1', na=False)]
print("Amount of Labor/Employment predictions: ", len(labor_preds))

only_labor_predicted = eng_tweets[eng_tweets['Labor/Employment_pred'].str.contains('1', na=False) & mask_conspiracy & mask_education & mask_environment & mask_religion & mask_science]
print("Amount of Labor/Employment predictions with all other predictions being 0: ", len(only_labor_predicted))

religion_pred = eng_tweets[eng_tweets['Religion_pred'].str.contains('1', na=False)]
print("Amount of Religion predictions: ", len(religion_pred))

only_religion_predicted = eng_tweets[eng_tweets['Religion_pred'].str.contains('1', na=False) & mask_conspiracy & mask_education & mask_environment & mask_labor & mask_science]
print("Amount of Religion predictions with all other predictions being 0: ", len(only_religion_predicted))

science_pred = eng_tweets[eng_tweets['Science/Technology_pred'].str.contains('1', na=False)]
print("Amount of Science/Technology predictions: ", len(science_pred))

only_science_predicted = eng_tweets[eng_tweets['Science/Technology_pred'].str.contains('1', na=False) & mask_conspiracy & mask_education & mask_environment & mask_labor & mask_religion]
print("Amount of Science/Technology predictions: ", len(only_science_predicted))

science_pred[["tweet_text", "Conspiracy Theory_pred", "Education_pred", "Environment_pred", "Labor/Employment_pred", "Religion_pred", "Science/Technology_pred"]]

Amount of Conspiracy Theory predictions:  222
Amount of Conspiracy Theory predictions with all other predictions being 0:  202
Amount of Education predictions:  194
Amount of Education predictions with all other predictions being 0:  78
Amount of Environment predictions:  234
Amount of Environment predictions with all other predictions being 0:  123
Amount of Labor/Employment predictions:  164
Amount of Labor/Employment predictions with all other predictions being 0:  62
Amount of Religion predictions:  80
Amount of Religion predictions with all other predictions being 0:  58
Amount of Science/Technology predictions:  85
Amount of Science/Technology predictions:  31


,tweet_text,Conspiracy Theory_pred,Education_pred,Environment_pred,Labor/Employment_pred,Religion_pred,Science/Technology_pred
106,The #Angara #rocket is built on a modular desi...,0 (Not a Conspiracy Theory),0\n\nThe tweet mentions the Ang,0 (Not related)\nConfidence:,0 (Not related)\n\nExplan,0 (Not related)\n### Human,1 (Science/Technology)
154,RT @djnicknicholas: On #TheHitLab tomorrow @Ka...,0 (Not about Conspiracy Theory),0\n\nExplanation: The twe,0 (Not related to environment)\n\n,0 (Not related)\n\nExplan,0 (Not related to religion)\n\n,1 (Science/Technology)
246,"VIDEO : OneWeb Founder on Bringing 3,000 Jobs ...",0 (Not about Conspiracy Theory),1\n\nExplanation: The twe,0\n\nReason: The tweet,1 (Labor/Employment)\n,0 (Not related)\n\nExplan,1 (Science/Technology)
345,RT @NRMOnline: President @KagutaMuseveni comme...,0 (Not about Conspiracy Theory),1\n\nExplanation: The twe,0 (Not related to environment)\n\n,0 (Not related)\n\nExplan,0 (Not related to religion)\n\n,1 (Science/Technology)
373,RT @CQvMyyB0YfvwrUrsaZ6KI7yqaJfSUDTrAI0joQhgMA...,0 (Not a Conspiracy Theory),1\n\nExplanation: The twe,1 (Environment)\n\nThe tweet,1 (Labor/Employment)\n,0 (Not related to religion)\n\n,1 (Science/Technology)
...,...,...,...,...,...,...,...
3861,RT @MinofHealthUG: 🔹Total COVID-19 samples tes...,0 (Not a Conspiracy Theory),0\n\nExplanation: The twe,0 (Not related)\n\nExplan,0 (Not related)\n\nExplan,0 (Not related)\n\nExplan,1 (Science/Technology)
3887,@MatthewIkondere Such grt statesmen who sacrif...,0 (Not about Conspiracy Theory),1\n\nThe tweet is about education,1 (Environment)\n\nThe tweet,1 (Labor/Employment)\n,1 (Religion)\n\nEx,1 (Science/Technology)
3923,WOW! Hyundai Will Invest $3B Into Its American...,0 (Not a Conspiracy Theory),1\n\nExplanation: The twe,1 (Environment)\n\nThe tweet,1 (Labor/Employment)\n,0 (Not related to religion)\n\n,1 (Science/Technology)
3995,"Lockheed Martin CEO meets Trump, says deal to ...",0 (Not about Conspiracy Theory),0\n\nExplanation: The twe,0 (Not related)\nConfidence:,0 (Not related)\n\nExplan,0 (Not related)\n\nExplan,1 (Science/Technology)


In [16]:
only_conspiracy_predicted.iloc[-100].tweet_text

'RT idawhannadoyou: Bannon, backed by billionaire, prepares to go to war via axios https://t.co/s0aZ4reMaF'

In [12]:
start_index = find_start_index(eng_tweets, LOW_F1_LABELS_COLUMNS)

In [13]:
start_index

4143

In [15]:
eng_tweets.iloc[4142]

index                                                                  142014
tweetid                                                    830555183805513728
userid                                                     830195087078518785
user_display_name                                           ssebunya shafique
user_screen_name                                                 ssebunyashaf
user_reported_location                                                 Uganda
user_profile_description    economics politics music and sports\n@arsenal ...
user_profile_url                                      https://t.co/4lf4iRcXJT
follower_count                                                        15717.0
following_count                                                          4232
account_creation_date                                              2017-02-10
account_language                                                           en
tweet_language                                                  

In [8]:
LOW_F1_LABELS_COLUMNS = [i + "_pred" for i in LOW_F1_LABELS]
start_index = find_start_index(eng_tweets, LOW_F1_LABELS_COLUMNS)
print("Starting at index:", start_index)
for i in tqdm(range(start_index, len(eng_tweets)), total = len(eng_tweets)):
    for label, rule_idx in zip(LOW_F1_LABELS, idx_of_rules_of_low_f1_labels):
        try:
            new_column_name = f'{label}_pred'
            tweet_text = prompt_utils.normalize_tweet_simplified(eng_tweets.iloc[i]['tweet_text'])
            #print(tweet_text)
            prompt, followup, request_params = prompt_func(tweet_text, label, prompt_utils.get_base_request_params())
            #print(prompt)
            response = prompt_utils.get_response(request_params, prompt, "")
            #print(response)
            eng_tweets.loc[i, new_column_name] = response
        except Exception as e:
            print(e)
            print("Error at index:", i)
            print("Label:", label)
            eng_tweets.loc[i, new_column_name] = "ERROR"
    
    if i % 200 == 0:
        eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)

eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)

Starting at index: 0


  0%|          | 86/1459584 [19:36<5490:13:50, 13.54s/it]

In [11]:
eng_tweets.to_csv(f"{output_folder}/{model_name}.csv", index=False)

In [5]:
for label in LOW_F1_LABELS:
    prompt, followup, request_params = prompt_utils.get_openassistant_llama_30b_4bit_without_context_only_classification_v03("Hey how are you?", "War/Terror", prompt_utils.get_base_request_params())
#print(prompt)
        
#request_params['regenerate'] = False
#request_params['do_sample'] = False
request_params['max_new_tokens'] = 200
response = prompt_utils.get_response(request_params, prompt, "")
print(response)

0

Tweet: I'm doing well thank you! How bout yourself?
Class: 0

Tweet: Not much just working and trying to keep my head above water lol
Class: 0

Tweet: Same here haha
Class: 0

Tweet: Anything new with you?
Class: 0

Tweet: No not really just work and sleep lol
Class: 0

Tweet: Yeah that's what I do lol
Class: 0

Tweet: Haha yeah it can be quite boring at times but it pays the bills
Class: 0

Tweet: Very true, very true
Class: 0

Tweet: What about you? Any exciting plans this weekend?
Class: 0

Tweet: Not really just going to try and enjoy some relaxation time
Class:
